# Identifying Lipid related genes using TCGA subtypes

This notebook creates a list by taking all proteins and comparing their abundance by grouping by Mesenchymal/ Normal and Mesenchymal/Proneural. The common genes between the two t-test were then searched to see if they were in Uniprot's list of genes related to Lipid metabolism 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
#sys.path.append('C:\\Users\\brittany henderson\\GitHub\\GBM_for_CPTAC\\')
#import cis_functions as f

import cptac
import cptac.utils as u


In [2]:
def add_significance_col(results_df, num_comparisons):
    "bonferroni multiple hypothesis"""
    alpha = .05
    bonferroni_cutoff = alpha / num_comparisons
    
    pval = results_df['P_Value']
    if float(pval[0]) <= bonferroni_cutoff:
        results_df['Significant'] = True
    else: 
        results_df['Significant'] = False
    return results_df

def wrap_ttest_return_all(df, label_column, comparison_columns, total_tests, alpha=.05):
    try:
        #Verify precondition that label column exists and has exactly 2 unique values
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        #Partition dataframe into two sets, one for each of the two unique values from the label column
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]
        
        #Determine the number of real valued columns on which we will do t-tests
        #sites = len(comparison_columns.columns)
        number_of_comparisons = total_tests # ? phospho sites or num freq mut genes doing cis comp
        
        #Use a bonferroni correction to adjust for multiple testing by altering the p-value needed for acceptance
        bonferroni_cutoff = alpha/number_of_comparisons
        
        #Store all comparisons with their p-values in a dictionary
        all_comparisons = {}
        
        #Loop through each comparison column, perform the t-test, and determine whether it meets the significance cutoff'''
        for column in comparison_columns:
            stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
            all_comparisons[column] = pval
    
        #Sort dictionary to list smallest p-values first
        sorted_comparisons = sorted(all_comparisons.items(), key=lambda kv: kv[1])
        #Format as a dataframe and return to caller
        all_comparisons_df = pd.DataFrame.from_dict(sorted_comparisons)
        all_comparisons_df.columns = ['Comparison', 'P_Value']
        
                                               
        all_comparisons_sig_col = add_significance_col(all_comparisons_df, number_of_comparisons)
        return all_comparisons_sig_col
                                
    except:
        print("Incorrectly Formatted Dataframe!")
        return None


In [3]:
#cptac.download(dataset='ccrcc', version='0.0')
brain= cptac.Gbm()
desired_cutoff = 0.05
gene = 'RB1'

In [31]:
#join clinical and proteomic data
clin_and_prot = brain.join_metadata_to_omics(metadata_df_name="clinical", omics_df_name="proteomics")
clin_and_prot = clin_and_prot.rename(columns = {"Patient_ID": "case"})
#clin_and_prot = clin_and_prot.reset_index()
clin_and_prot

Name,case,Sample_Tumor_Normal,age,gender,height,weight,bmi,country_of_origin,race,ethnicity,...,ZSCAN31_proteomics,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,C3L-00104,Tumor,58.0,Male,188.00,115.00,32.54,United States,White,Not-Hispanic or Latino,...,NaN,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990
S002,C3L-00365,Tumor,59.0,Female,162.00,54.00,20.61,United States,White,Not-Hispanic or Latino,...,NaN,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037
S003,C3L-00674,Tumor,45.0,Male,193.00,102.00,27.44,NaN,White,Not-Hispanic or Latino,...,NaN,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581
S004,C3L-00677,Tumor,69.0,Female,164.00,52.00,19.32,NaN,White,Not-Hispanic or Latino,...,-0.062127,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715
S005,C3L-01040,Tumor,77.0,Female,170.00,70.00,24.22,Russia,NaN,NaN,...,NaN,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S111,PT-RN5K,Normal,56.0,Female,162.56,49.90,18.88,NaN,White,Not Hispanic or Latino,...,NaN,0.298664,-0.403930,-0.482597,0.240966,NaN,0.609314,-1.293464,0.225910,-0.013637
S112,PT-RU72,Normal,59.0,Female,162.56,63.05,23.86,NaN,Asian,NaN,...,NaN,0.155988,0.002551,-0.834434,NaN,-0.073908,0.637394,-0.795886,0.326046,-0.217014
S113,PT-UTHO,Normal,68.0,Male,182.88,62.87,18.80,NaN,White,NaN,...,NaN,0.016857,-0.171858,-0.462233,NaN,1.278683,0.616285,-0.734952,0.339283,-0.419138


In [33]:
#Read in files with TCGA subtypes
subtypes = pd.read_csv("/Users/Lindsey/Downloads/gbm_all_subtype_collections.2019-11-13.tsv", sep= "\t")

case_subtype = subtypes[["case",'rna_wang_cancer_cell_2017']] #only need subtype and case
case_subtype = case_subtype.rename(columns = {"rna_wang_cancer_cell_2017": "TCGA_subtype"})


subtypes.head()


,case,sample_type,nmf_consensus,nmf_cluster_membership,rna_wang_cancer_cell_2017,mRNA_stemness_index,dna_methyl,is_gcimp,immune,telomere,lipid,mirna,ancestry_prediction,ancestry_prediction_afr_prob,ancestry_prediction_amr_prob,ancestry_prediction_eas_prob,ancestry_prediction_eur_prob,ancestry_prediction_sas_prob,wxs_total_mutation,wgs_total_mutation
0,C3L-00104,tumor,nmf1,0.743,Proneural,0.678244,dm2,True,low,normal,NaN,mi5,EUR,0.00,0.03,0.00,0.97,0.00,60.0,2632.0
1,C3L-00365,tumor,nmf3,0.614,Classical,0.681122,dm4,False,low,normal,TAG_enriched,mi1,EUR,0.00,0.02,0.00,0.98,0.00,57.0,7628.0
2,C3L-00674,tumor,nmf1,0.507,Mesenchymal,0.744635,dm5,False,high,normal,TAG_enriched,mi3,EUR,0.01,0.00,0.01,0.98,0.00,37.0,1233.0
3,C3L-00677,tumor,nmf1,0.536,Proneural,0.900896,dm5,False,low,long,TAG_enriched,mi5,EUR,0.02,0.11,0.00,0.85,0.02,925.0,16955.0
4,C3L-01040,tumor,nmf1,0.589,Classical,0.647288,dm5,False,low,normal,NaN,mi1,EUR,0.00,0.03,0.00,0.97,0.00,85.0,4298.0


In [34]:
case_subtype = case_subtype.replace(np.nan, 'normal', regex=True)
case_subtype = case_subtype.set_index("case")

case_subtype


,TCGA_subtype
case,
C3L-00104,Proneural
C3L-00365,Classical
C3L-00674,Mesenchymal
C3L-00677,Proneural
C3L-01040,Classical
...,...
PT-RN5K,normal
PT-RU72,normal
PT-UTHO,normal


In [35]:
# merge tgca subtypes with proteomics and clincal df
prot_subtype= clin_and_prot.merge(case_subtype, on='case')

prot_subtype = prot_subtype.set_index("case")

prot_subtype

,Sample_Tumor_Normal,age,gender,height,weight,bmi,country_of_origin,race,ethnicity,ethnicity_self_identify,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,TCGA_subtype
case,,,,,,,,,,,,,,,,,,,,,
C3L-00104,Tumor,58.0,Male,188.00,115.00,32.54,United States,White,Not-Hispanic or Latino,Caucasian,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Proneural
C3L-00365,Tumor,59.0,Female,162.00,54.00,20.61,United States,White,Not-Hispanic or Latino,Caucasian,...,0.161975,-0.213093,0.235571,NaN,0.107421,0.048724,0.138403,-0.290141,0.405037,Classical
C3L-00674,Tumor,45.0,Male,193.00,102.00,27.44,NaN,White,Not-Hispanic or Latino,Caucasian,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Mesenchymal
C3L-00677,Tumor,69.0,Female,164.00,52.00,19.32,NaN,White,Not-Hispanic or Latino,Caucasian,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Proneural
C3L-01040,Tumor,77.0,Female,170.00,70.00,24.22,Russia,NaN,NaN,Caucasian,...,-0.092502,0.010639,-0.465079,NaN,-0.500083,0.112651,1.004660,-0.230304,-0.102416,Classical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PT-RN5K,Normal,56.0,Female,162.56,49.90,18.88,NaN,White,Not Hispanic or Latino,NaN,...,0.298664,-0.403930,-0.482597,0.240966,NaN,0.609314,-1.293464,0.225910,-0.013637,normal
PT-RU72,Normal,59.0,Female,162.56,63.05,23.86,NaN,Asian,NaN,NaN,...,0.155988,0.002551,-0.834434,NaN,-0.073908,0.637394,-0.795886,0.326046,-0.217014,normal
PT-UTHO,Normal,68.0,Male,182.88,62.87,18.80,NaN,White,NaN,NaN,...,0.016857,-0.171858,-0.462233,NaN,1.278683,0.616285,-0.734952,0.339283,-0.419138,normal


# Mesenchymal VS Normal

In [36]:
#create mesenchymal and normal only df
Mesenchymal = (prot_subtype.loc[prot_subtype['TCGA_subtype'] == 'Mesenchymal'])
Normal = (prot_subtype.loc[prot_subtype['Sample_Tumor_Normal'] == 'Normal'])
Normal.head()

,Sample_Tumor_Normal,age,gender,height,weight,bmi,country_of_origin,race,ethnicity,ethnicity_self_identify,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,TCGA_subtype
case,,,,,,,,,,,,,,,,,,,,,
PT-NPJ7,Normal,68.0,Female,160.02,74.84,29.23,NaN,White,NaN,NaN,...,0.337310,0.019538,-1.375146,NaN,0.164100,0.767921,-1.128100,0.209480,-0.205739,normal
PT-P44H,Normal,43.0,Male,177.80,112.04,35.44,NaN,White,NaN,NaN,...,0.335445,-0.199247,-0.798595,NaN,0.517515,0.379683,-1.087567,0.210812,-0.176034,normal
PT-Q2AG,Normal,42.0,Female,177.80,102.06,32.28,NaN,White,NaN,NaN,...,0.252516,-0.102542,NaN,NaN,NaN,0.791386,-0.667928,0.053734,-0.229870,normal
PT-QVJO,Normal,64.0,Female,170.18,55.79,19.26,NaN,White,Not Hispanic or Latino,NaN,...,0.320193,-0.098267,-1.321314,NaN,-1.330917,0.667393,-1.172195,0.303740,-0.556137,normal
PT-R55F,Normal,55.0,Male,177.80,77.11,24.39,NaN,White,Not Hispanic or Latino,NaN,...,0.326426,-0.109841,-0.676704,NaN,0.080553,0.563370,-0.973314,0.478914,-0.415546,normal


In [38]:
# create mesenchymal and normal df 
Mesench_Normal = Mesenchymal.append(Normal)
Mesench_Normal.drop(Mesench_Normal.iloc[:, :28], axis=1, inplace=True)


In [41]:
#create gene list for t-test
prot_col_list = list(Mesench_Normal.columns)
prot_col_list.remove('TCGA_subtype')

prot_col_list


['A2M_proteomics',
 'AAAS_proteomics',
 'AACS_proteomics',
 'AADAT_proteomics',
 'AAED1_proteomics',
 'AAGAB_proteomics',
 'AAK1_proteomics',
 'AAMDC_proteomics',
 'AAMP_proteomics',
 'AAR2_proteomics',
 'AARS_proteomics',
 'AARS2_proteomics',
 'AARSD1_proteomics',
 'AASDHPPT_proteomics',
 'AASS_proteomics',
 'AATF_proteomics',
 'AATK_proteomics',
 'ABAT_proteomics',
 'ABCA1_proteomics',
 'ABCA2_proteomics',
 'ABCA3_proteomics',
 'ABCA5_proteomics',
 'ABCA8_proteomics',
 'ABCB1_proteomics',
 'ABCB10_proteomics',
 'ABCB6_proteomics',
 'ABCB7_proteomics',
 'ABCB8_proteomics',
 'ABCB9_proteomics',
 'ABCC1_proteomics',
 'ABCC3_proteomics',
 'ABCC4_proteomics',
 'ABCD1_proteomics',
 'ABCD2_proteomics',
 'ABCD3_proteomics',
 'ABCD4_proteomics',
 'ABCE1_proteomics',
 'ABCF1_proteomics',
 'ABCF2_proteomics',
 'ABCF3_proteomics',
 'ABCG1_proteomics',
 'ABCG2_proteomics',
 'ABHD10_proteomics',
 'ABHD11_proteomics',
 'ABHD12_proteomics',
 'ABHD13_proteomics',
 'ABHD14A-ACY1_proteomics',
 'ABHD14B

In [42]:
#Call wrap_ttest, pass in formatted dataframe

prot_all_comparisons = u.wrap_ttest(Mesench_Normal, 'TCGA_subtype', prot_col_list)
prot_num_comparisons = len(prot_col_list)
print("Number of comparisons:", prot_num_comparisons)
prot_bonferroni_cutoff = .05 / prot_num_comparisons
print("Bonferroni cutoff = ", prot_bonferroni_cutoff)
print("Logged Bonferroni cutoff = ", np.log10(prot_bonferroni_cutoff))


5291 significant comparisons!
Number of comparisons: 11140
Bonferroni cutoff =  4.4883303411131066e-06
Logged Bonferroni cutoff =  -5.347915186501691


In [43]:
#print significant comparisons 
prot_all_comparisons = prot_all_comparisons.dropna(axis=0)
prot_sig_comparisons = prot_all_comparisons.loc[prot_all_comparisons['P_Value'] <= prot_bonferroni_cutoff]
print("Number of significant Proteomics comparisons: ", len(prot_sig_comparisons), '\n')

if len(prot_sig_comparisons) > 0:
    print(prot_sig_comparisons)

prot_sig_comparisons_Mesench_Normal = prot_sig_comparisons

Number of significant Proteomics comparisons:  5291 

               Comparison       P_Value
0        PI4KA_proteomics  1.106746e-35
1         WDR7_proteomics  2.413637e-33
2       ANKS1B_proteomics  4.412780e-33
3        DMXL2_proteomics  1.011996e-32
4         MADD_proteomics  2.561057e-32
...                   ...           ...
5286  C12orf73_proteomics  4.344286e-06
5287     KDM4A_proteomics  4.350479e-06
5288      AGO2_proteomics  4.353342e-06
5289    ZNF618_proteomics  4.381527e-06
5290     CYTH4_proteomics  4.432903e-06

[5291 rows x 2 columns]


In [44]:
prot_sig_comparisons_Mesench_Normal.set_index("Comparison")
prot_sig_comparisons_Mesench_Normal = prot_sig_comparisons_Mesench_Normal.rename(columns = {"P_Value": "P_Value_MN"})
prot_sig_comparisons_Mesench_Normal

,Comparison,P_Value_MN
0,PI4KA_proteomics,1.106746e-35
1,WDR7_proteomics,2.413637e-33
2,ANKS1B_proteomics,4.412780e-33
3,DMXL2_proteomics,1.011996e-32
4,MADD_proteomics,2.561057e-32
...,...,...
5286,C12orf73_proteomics,4.344286e-06
5287,KDM4A_proteomics,4.350479e-06
5288,AGO2_proteomics,4.353342e-06
5289,ZNF618_proteomics,4.381527e-06


# Mesenchymal and Proneural

In [45]:
#create df that are only Mesenchymal and Proneural 
Mesenchymal = (prot_subtype.loc[prot_subtype['TCGA_subtype'] == 'Mesenchymal'])
Proneural = (prot_subtype.loc[prot_subtype['TCGA_subtype'] == 'Proneural'])
Proneural.head()

,Sample_Tumor_Normal,age,gender,height,weight,bmi,country_of_origin,race,ethnicity,ethnicity_self_identify,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,TCGA_subtype
case,,,,,,,,,,,,,,,,,,,,,
C3L-00104,Tumor,58.0,Male,188.0,115.0,32.54,United States,White,Not-Hispanic or Latino,Caucasian,...,-0.047437,-0.105908,-0.347076,NaN,0.459635,0.079452,-0.784983,-0.488441,0.167990,Proneural
C3L-00677,Tumor,69.0,Female,164.0,52.0,19.32,NaN,White,Not-Hispanic or Latino,Caucasian,...,-0.254535,0.463653,0.580230,0.503044,-0.604986,0.178077,-0.720059,-0.150197,-0.268715,Proneural
C3L-01146,Tumor,59.0,Male,175.0,96.0,31.35,Russia,NaN,NaN,Caucasian,...,-0.176744,0.134078,-0.143252,NaN,-0.425889,-0.008761,0.277088,-0.296844,-0.054669,Proneural
C3L-01149,Tumor,48.0,Male,173.0,91.0,30.41,Russia,NaN,NaN,Caucasian,...,-0.191377,0.402318,2.018452,NaN,0.504515,0.732463,-0.613257,-0.115444,0.566719,Proneural
C3L-01157,Tumor,66.0,Female,162.0,75.0,28.58,Russia,NaN,NaN,Caucasian,...,-0.059525,0.087678,-0.080362,0.013261,0.472166,0.335460,0.150904,-0.010514,-0.230476,Proneural


In [46]:
#create Mesenchymal and Proneural df 
Mesench_Proneural = Mesenchymal.append(Proneural)
Mesench_Proneural.drop(Mesench_Proneural.iloc[:, :28], axis=1, inplace=True)

Mesench_Proneural

,A2M_proteomics,AAAS_proteomics,AACS_proteomics,AADAT_proteomics,AAED1_proteomics,AAGAB_proteomics,AAK1_proteomics,AAMDC_proteomics,AAMP_proteomics,AAR2_proteomics,...,ZSWIM8_proteomics,ZW10_proteomics,ZWILCH_proteomics,ZWINT_proteomics,ZXDC_proteomics,ZYG11B_proteomics,ZYX_proteomics,ZZEF1_proteomics,ZZZ3_proteomics,TCGA_subtype
case,,,,,,,,,,,,,,,,,,,,,
C3L-00674,1.096470,-0.094421,-0.106304,0.084578,0.176402,-0.248151,0.014061,-0.699773,-0.638462,0.592874,...,-0.065534,-0.306717,0.879991,NaN,0.883564,-0.172222,0.011876,-0.131889,-0.503581,Mesenchymal
C3L-01043,0.769231,-0.011039,-0.152467,-0.457411,0.679201,-0.106294,-0.428006,-0.781859,0.039258,-0.080868,...,-0.026241,0.118676,0.444517,NaN,0.835224,-0.271005,0.371339,-0.168640,0.144939,Mesenchymal
C3L-01045,-0.735991,0.125775,-0.285444,-0.620125,NaN,0.166882,-0.110721,-0.749020,-0.041834,0.601213,...,0.033878,0.051231,0.211483,-0.287100,-0.230124,-0.047399,0.761362,-0.376203,0.476902,Mesenchymal
C3L-01046,-0.037553,0.239725,0.152238,-0.505107,NaN,-0.133864,0.191524,0.030741,-0.036606,0.115695,...,0.176411,0.003886,0.720046,-0.295464,-0.543543,0.118416,0.963697,0.243561,0.013539,Mesenchymal
C3L-01049,-0.310086,0.087263,-0.163304,NaN,-0.009380,0.091515,-0.393903,-0.217689,-0.195168,0.302267,...,-0.053609,0.060704,0.635137,NaN,0.199433,-0.340400,0.271082,-0.122117,0.177207,Mesenchymal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-02785,0.691145,0.369580,0.196114,0.028247,0.878197,-0.152089,0.457016,0.017247,-0.838311,0.022492,...,0.144039,0.046498,0.667878,NaN,-0.068733,-0.198584,-0.307852,-0.092755,0.376837,Proneural
C3N-02788,-0.340301,-0.381604,0.176850,0.056150,-0.381644,-0.249617,0.737844,0.114329,-0.200811,-0.477411,...,0.196207,-0.076559,-0.723503,-0.915281,0.171156,-0.126355,-0.806935,0.017285,0.148011,Proneural
C3N-03088,0.836600,-0.519413,0.408439,0.150180,NaN,-0.201109,1.047781,0.235729,-0.264215,-0.348418,...,0.008570,-0.247399,-0.764748,-0.347858,-0.133599,-0.148703,0.000129,0.253013,0.284701,Proneural


In [47]:
#create list of genes to pass into t-test
prot_col_list = list(Mesench_Proneural.columns)
prot_col_list.remove('TCGA_subtype')

prot_col_list

['A2M_proteomics',
 'AAAS_proteomics',
 'AACS_proteomics',
 'AADAT_proteomics',
 'AAED1_proteomics',
 'AAGAB_proteomics',
 'AAK1_proteomics',
 'AAMDC_proteomics',
 'AAMP_proteomics',
 'AAR2_proteomics',
 'AARS_proteomics',
 'AARS2_proteomics',
 'AARSD1_proteomics',
 'AASDHPPT_proteomics',
 'AASS_proteomics',
 'AATF_proteomics',
 'AATK_proteomics',
 'ABAT_proteomics',
 'ABCA1_proteomics',
 'ABCA2_proteomics',
 'ABCA3_proteomics',
 'ABCA5_proteomics',
 'ABCA8_proteomics',
 'ABCB1_proteomics',
 'ABCB10_proteomics',
 'ABCB6_proteomics',
 'ABCB7_proteomics',
 'ABCB8_proteomics',
 'ABCB9_proteomics',
 'ABCC1_proteomics',
 'ABCC3_proteomics',
 'ABCC4_proteomics',
 'ABCD1_proteomics',
 'ABCD2_proteomics',
 'ABCD3_proteomics',
 'ABCD4_proteomics',
 'ABCE1_proteomics',
 'ABCF1_proteomics',
 'ABCF2_proteomics',
 'ABCF3_proteomics',
 'ABCG1_proteomics',
 'ABCG2_proteomics',
 'ABHD10_proteomics',
 'ABHD11_proteomics',
 'ABHD12_proteomics',
 'ABHD13_proteomics',
 'ABHD14A-ACY1_proteomics',
 'ABHD14B

In [48]:
#Call wrap_ttest, pass in formatted dataframe

prot_all_comparisons = u.wrap_ttest(Mesench_Proneural, 'TCGA_subtype', prot_col_list)
prot_num_comparisons = len(prot_col_list)
print("Number of comparisons:", prot_num_comparisons)
prot_bonferroni_cutoff = .05 / prot_num_comparisons
print("Bonferroni cutoff = ", prot_bonferroni_cutoff)
print("Logged Bonferroni cutoff = ", np.log10(prot_bonferroni_cutoff))

961 significant comparisons!
Number of comparisons: 11140
Bonferroni cutoff =  4.4883303411131066e-06
Logged Bonferroni cutoff =  -5.347915186501691


In [49]:
#Print significant comparisons 
prot_all_comparisons = prot_all_comparisons.dropna(axis=0)
prot_sig_comparisons = prot_all_comparisons.loc[prot_all_comparisons['P_Value'] <= prot_bonferroni_cutoff]
print("Number of significant Proteomics comparisons: ", len(prot_sig_comparisons), '\n')

if len(prot_sig_comparisons) > 0:
    print(prot_sig_comparisons)
    
prot_sig_comparisons_Mesench_Proneural = prot_sig_comparisons

Number of significant Proteomics comparisons:  961 

             Comparison       P_Value
0     PODXL2_proteomics  2.047362e-14
1      BASP1_proteomics  2.654212e-14
2     GPRIN1_proteomics  7.094843e-14
3      PHF24_proteomics  7.994311e-14
4       SCAI_proteomics  1.492919e-13
..                  ...           ...
956  CNTNAP1_proteomics  4.392254e-06
957    HSPB1_proteomics  4.421559e-06
958    NOVA1_proteomics  4.426351e-06
959   THSD7A_proteomics  4.466200e-06
960   PARP14_proteomics  4.483369e-06

[961 rows x 2 columns]


In [50]:
prot_sig_comparisons_Mesench_Proneural.set_index("Comparison")
prot_sig_comparisons_Mesench_Proneural = prot_sig_comparisons_Mesench_Proneural.rename(columns = {"P_Value": "P_Value_MP"})
prot_sig_comparisons_Mesench_Proneural

,Comparison,P_Value_MP
0,PODXL2_proteomics,2.047362e-14
1,BASP1_proteomics,2.654212e-14
2,GPRIN1_proteomics,7.094843e-14
3,PHF24_proteomics,7.994311e-14
4,SCAI_proteomics,1.492919e-13
...,...,...
956,CNTNAP1_proteomics,4.392254e-06
957,HSPB1_proteomics,4.421559e-06
958,NOVA1_proteomics,4.426351e-06
959,THSD7A_proteomics,4.466200e-06


In [51]:
#create list of common significant genes
common_sig = prot_sig_comparisons_Mesench_Proneural.merge(prot_sig_comparisons_Mesench_Normal, on='Comparison')
common_sig.replace(to_replace = '_proteomics', value = '', 
                        inplace = True, regex = True)# shorten column names


sig_list  = common_sig['Comparison'] 
sig_list

0       PODXL2
1        BASP1
2       GPRIN1
3        PHF24
4         SCAI
        ...   
909       CD47
910    CNTNAP1
911      HSPB1
912     THSD7A
913     PARP14
Name: Comparison, Length: 914, dtype: object

In [52]:
#Calculate mean
Mesenchymal["MTMR1_proteomics"].mean()


-0.20449052751586466

In [53]:
#check to see if mean is up in normal
Normal["MTMR1_proteomics"].mean()

0.9241884660140893

In [54]:
#read in uniProt file for lipid metabolism 
lipid_genes = pd.read_csv("/Users/Lindsey/Downloads/uniprot_keyword_Lipid.tab", sep= "\t")
lipid_genes



,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
0,Q14032,BAAT_HUMAN,reviewed,Bile acid-CoA:amino acid N-acyltransferase (BA...,BAAT,Homo sapiens (Human),418
1,P55084,ECHB_HUMAN,reviewed,"Trifunctional enzyme subunit beta, mitochondri...",HADHB MSTP029,Homo sapiens (Human),474
2,Q3SYC2,MOGT2_HUMAN,reviewed,2-acylglycerol O-acyltransferase 2 (EC 2.3.1.2...,MOGAT2 DC5 DGAT2L5,Homo sapiens (Human),334
3,Q15800,MSMO1_HUMAN,reviewed,Methylsterol monooxygenase 1 (EC 1.14.18.9) (C...,MSMO1 DESP4 ERG25 SC4MOL,Homo sapiens (Human),293
4,O60494,CUBN_HUMAN,reviewed,Cubilin (460 kDa receptor) (Intestinal intrins...,CUBN IFCR,Homo sapiens (Human),3623
...,...,...,...,...,...,...,...
480,Q49AA0,ZFP69_HUMAN,reviewed,Zinc finger protein 69 homolog (Zinc finger pr...,ZFP69 ZNF642,Homo sapiens (Human),526
481,O95477,ABCA1_HUMAN,reviewed,Phospholipid-transporting ATPase ABCA1 (EC 7.6...,ABCA1 ABC1 CERP,Homo sapiens (Human),2261
482,O94905,ERLN2_HUMAN,reviewed,Erlin-2 (Endoplasmic reticulum lipid raft-asso...,ERLIN2 C8orf2 SPFH2 UNQ2441/PRO5003/PRO9924,Homo sapiens (Human),339
483,A1L3X0,ELOV7_HUMAN,reviewed,Elongation of very long chain fatty acids prot...,ELOVL7,Homo sapiens (Human),281


In [55]:
# create list of significant genes (common in both t-tests) that are also in Uniprot df 
genes_dirty = list(lipid_genes['Gene names'])
genes_clean = []
for e in genes_dirty:
    bits = e.split(" ")
    genes_clean.append(bits[0])
    
 
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

winners = intersection(sig_list, genes_clean)
#print (winners)
#list from proteomics correlation
winners

['PLCB1',
 'B4GALT5',
 'MTMR1',
 'SULT4A1',
 'SLC27A4',
 'ACOT7',
 'PLBD1',
 'PTPRN2',
 'BSCL2',
 'PCYT1A',
 'SMPD3',
 'DAGLA',
 'PLCG2',
 'HDLBP']